In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações visuais para os gráficos
sns.set_theme(style="whitegrid")
np.random.seed(42) # Garante que os resultados aleatórios sejam os mesmos toda vez que rodar
tamanho_amostra = 100000 # Gerar muitos pontos para ter uma curva suave

# --- GRÁFICO 1: VARIANDO O PARÂMETRO μ (mu), O "CENTRO DE GRAVIDADE" ---

# Mantemos o sigma (espalhamento) fixo para ver apenas o efeito do mu
sigma_fixo = 0.5
mus_variaveis = [1.0, 2.0, 3.0]

# Cria uma figura com 3 gráficos lado a lado
fig, axes = plt.subplots(1, 3, figsize=(18, 5), sharey=True)
fig.suptitle('Efeito da Variação de μ (Centro de Gravidade) com σ Fixo', fontsize=16, y=1.02)

for i, mu in enumerate(mus_variaveis):
    # Gera 100.000 números aleatórios seguindo a distribuição Log-Normal com os parâmetros
    amostra = np.random.lognormal(mean=mu, sigma=sigma_fixo, size=tamanho_amostra)
    
    # Plota o histograma (barras) e a curva de densidade (a "rampa de skate")
    sns.histplot(amostra, bins=100, kde=True, ax=axes[i], line_kws={'linewidth': 3, 'color': 'red'})
    axes[i].set_title(f'μ = {mu:.1f} | σ = {sigma_fixo:.1f}')
    axes[i].set_xlabel('Valor Gerado')
    if i == 0:
        axes[i].set_ylabel('Frequência')

plt.show()


# --- GRÁFICO 2: VARIANDO O PARÂMETRO σ (sigma), O "FATOR ESPALHAMENTO" ---

# Mantemos o mu (centro) fixo para ver apenas o efeito do sigma
mu_fixo = 2.0
sigmas_variaveis = [0.2, 0.7, 1.5]

# Cria uma segunda figura com 3 gráficos
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Efeito da Variação de σ (Fator Espalhamento) com μ Fixo', fontsize=16, y=1.02)

for i, sigma in enumerate(sigmas_variaveis):
    # Gera os dados
    amostra = np.random.lognormal(mean=mu_fixo, sigma=sigma, size=tamanho_amostra)
    
    # Plota o gráfico
    sns.histplot(amostra, bins=100, kde=True, ax=axes[i], line_kws={'linewidth': 3, 'color': 'red'})
    axes[i].set_title(f'μ = {mu_fixo:.1f} | σ = {sigma:.1f}')
    axes[i].set_xlabel('Valor Gerado')
    if i == 0:
        axes[i].set_ylabel('Frequência')

plt.show()

In [0]:
# Converte o resultado da sua query Spark para um DataFrame Pandas
df_comparativo_pd = df_comparativo.toPandas()
df_comparativo_pd = df_comparativo_pd.sort_values('mes')

# Inicia a criação do gráfico
fig, ax1 = plt.subplots(figsize=(14, 8))
plt.title('Comparativo de Tendência: Volume de Transações Real vs. Sintético', fontsize=16)

# --- Linha para os Dados REAIS (Eixo da Esquerda) ---
cor_real = 'tab:blue'
ax1.set_xlabel('Mês do Ano')
ax1.set_ylabel('Volume REAL de Transações', color=cor_real, fontsize=12)
ax1.plot(df_comparativo_pd['mes'], df_comparativo_pd['total_tx_pf_pagador_REAL'], color=cor_real, marker='o', label='Volume Real')
ax1.tick_params(axis='y', labelcolor=cor_real)

# --- Linha para os Dados SINTÉTICOS (Eixo da Direita) ---
# Cria um segundo eixo Y que compartilha o mesmo eixo X
ax2 = ax1.twinx()  
cor_sintetico = 'tab:orange'
ax2.set_ylabel('Volume SINTÉTICO de Transações', color=cor_sintetico, fontsize=12)
ax2.plot(df_comparativo_pd['mes'], df_comparativo_pd['total_tx_pf_pagador_SINTETICO'], color=cor_sintetico, marker='x', linestyle='--', label='Volume Sintético')
ax2.tick_params(axis='y', labelcolor=cor_sintetico)

# Adiciona a legenda
fig.legend(loc="upper right", bbox_to_anchor=(0.9, 0.9))
plt.grid(True)
plt.show()

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# --- 1. Geração de Dados de Exemplo ---
# (Você pode pular esta parte e usar seu DataFrame real)
# Nós simulamos o "Ataque de Madrugada" (p_madrugada=0.70)
# e transações legítimas com pico às 14h (loc=14).

print("Gerando dados de exemplo...")

def criar_dados_exemplo(n_legit=50000, n_fraude=5000, p_madrugada=0.7):
    # Gerar horas para transações legítimas (pico em horário comercial)
    horas_legitimas = np.random.normal(loc=14, scale=4.0, size=n_legit)
    # Garantir que as horas estão no intervalo [0, 23]
    horas_legitimas = np.clip(horas_legitimas, 0, 23).astype(int)
    
    # Gerar horas para transações fraudulentas
    n_madrugada = int(n_fraude * p_madrugada)
    n_fraude_normal = n_fraude - n_madrugada
    
    # Fraudes de "Ataque de Madrugada" (1h-4h)
    # np.random.randint(1, 5) gera números inteiros: 1, 2, 3, 4
    horas_fraude_madrugada = np.random.randint(1, 5, size=n_madrugada)
    
    # Outras fraudes que seguem o padrão normal
    horas_fraude_normal = np.random.normal(loc=14, scale=4.0, size=n_fraude_normal)
    horas_fraude_normal = np.clip(horas_fraude_normal, 0, 23).astype(int)
    
    horas_fraude = np.concatenate([horas_fraude_madrugada, horas_fraude_normal])
    
    # Criar DataFrames
    df_legit = pd.DataFrame({'hora': horas_legitimas, 'is_fraud': 0})
    df_fraude = pd.DataFrame({'hora': horas_fraude, 'is_fraud': 1})
    
    # Combinar e embaralhar
    df_transacoes = pd.concat([df_legit, df_fraude]).sample(frac=1).reset_index(drop=True)
    
    # Adicionar uma coluna 'data' apenas para ser fiel ao schema
    # (A hora é a única coisa que importa para este gráfico)
    base_date = datetime(2023, 1, 1)
    df_transacoes['data'] = df_transacoes['hora'].apply(
        lambda h: base_date + timedelta(hours=int(h), minutes=np.random.randint(0, 60))
    )
    
    return df_transacoes

# Gerar os dados de exemplo
# No seu caso, você carregaria seu DataFrame de transações aqui
df_transacoes_exemplo = criar_dados_exemplo()

print(df_transacoes_exemplo.head())


# --- 2. Código para Geração da Visualização ---
# (Assumindo que você tem um DataFrame chamado 'df_transacoes'
# com as colunas 'data' [datetime] e 'is_fraud' [0 ou 1])

# Para o exemplo, usaremos 'df_transacoes_exemplo'
df_para_plotar = df_transacoes_exemplo

# **Passo A: Extrair a hora da coluna 'data'**
# (Seu DataFrame 'df_transacoes' real pode já ter a coluna 'hora' 
# do passo de geração de dados, mas caso contrário, crie-a assim)
if 'hora' not in df_para_plotar.columns:
    df_para_plotar['hora'] = df_para_plotar['data'].dt.hour

# **Passo B: Plotar o gráfico**
print("Gerando visualização...")

# Configurar o estilo do gráfico
sns.set_theme(style="whitegrid")
plt.figure(figsize=(16, 8))

# Criar o histograma com seaborn
# - 'x' é a coluna da hora
# - 'hue' é a coluna que separa as cores (fraude vs. legítima)
# - 'bins=24' cria uma barra para cada hora
# - 'multiple="stack"' empilha as barras de fraude sobre as legítimas
viz = sns.histplot(
    data=df_para_plotar,
    x='hora',
    hue='is_fraud',
    bins=24,
    multiple="stack",
    palette={0: "cornflowerblue", 1: "red"}, # 0=Legítima, 1=Fraude
    edgecolor="white",
    linewidth=0.5
)

# **Passo C: Melhorar os rótulos e a legenda**

# Mapear a legenda para nomes mais claros
try:
    handles = viz.legend_.legendHandles
    viz.legend_.remove()
    viz.legend(handles, ['Legítima', 'Fraude'], title='Tipo de Transação')
except AttributeError:
    # Caso a legenda não seja gerada automaticamente
    pass

# Configurar rótulos e título
plt.title('Distribuição de Transações por Hora do Dia (Legítimas vs. Fraudes)', fontsize=16, weight='bold')
plt.xlabel('Hora do Dia (0-23)', fontsize=12)
plt.ylabel('Contagem de Transações', fontsize=12)

# Força a exibição de todas as 24 horas no eixo X
plt.xticks(range(0, 24))
plt.xlim(-0.5, 23.5) # Ajusta os limites para centralizar as barras

# Exibir o gráfico
plt.show()

## Rastreio de Transações Fraudulentas

In [0]:


%sql
select 
 c.id as id_conta_origem,
case
  when startswith (cl.nome, 'Sr. ') then replace (cl.nome,'Sr. ','') 
  when startswith (cl.nome, 'Sra. ') then replace (cl.nome,'Sra. ','')
  when startswith (cl.nome, 'Srta. ') then replace (cl.nome,'Srta. ','')
  when startswith (cl.nome, 'Dra. ') then replace (cl.nome,'Dra. ', '')
  when startswith (cl.nome, 'Dr. ') then replace (cl.nome,'Dr. ','')
  else cl.nome 
  end as nome_remetente, 
  t.valor as valor_enviado,
  t.id_conta_destino,
  case
  when startswith (cl2.nome, 'Sr. ') then replace (cl2.nome,'Sr. ','') 
  when startswith (cl2.nome, 'Sra. ') then replace (cl2.nome,'Sra. ','')
  when startswith (cl2.nome, 'Srta. ') then replace (cl2.nome,'Srta. ','')
  when startswith (cl2.nome, 'Dra. ') then replace (cl2.nome,'Dra. ', '')
  when startswith (cl2.nome, 'Dr. ') then replace (cl2.nome,'Dr. ','')
  else cl2.nome 
  end as nome_destinatario_normalizado,
  t.id as id_transacao,
  t.data,
  t.is_fraud,
  t.fraud_type
from transacoes_db.copper.clientes as cl
join transacoes_db.copper.contas as c
on cl.id = c.id_cliente
join transacoes_db.copper.transacoes as t
on c.id = t.id_conta_origem
join transacoes_db.copper.contas as c2
on t.id_conta_destino = c2.id
join transacoes_db.copper.clientes as cl2 -- join para chamar novamente a nome mas através do relacionamento cl2.id
on c2.id_cliente = cl2.id









In [0]:
%sql
--Não está retornando as contas de destino de algunas fraudadores 
select 
id_conta_destino,
nome
from transacoes_db.copper.transacoes as t
join transacoes_db.copper.contas as c
on t.id_conta_destino = c.id
join transacoes_db.copper.clientes as cli
on c.id_cliente = cli.id
where id_conta_destino in (
'04ab576a-83e6-4b3d-bafc-afcb9cce0eb8')


## Total de Fraudes

In [0]:
%sql
select 
 c.id as id_conta_origem,
case
  when startswith (cl.nome, 'Sr. ') then replace (cl.nome,'Sr. ','') 
  when startswith (cl.nome, 'Sra. ') then replace (cl.nome,'Sra. ','')
  when startswith (cl.nome, 'Srta. ') then replace (cl.nome,'Srta. ','')
  when startswith (cl.nome, 'Dra. ') then replace (cl.nome,'Dra. ', '')
  when startswith (cl.nome, 'Dr. ') then replace (cl.nome,'Dr. ','')
  else cl.nome 
  end as nome_normalizado, 
  t.valor as valor_enviado,
  t.id_conta_destino,
  case
  when startswith (cl2.nome, 'Sr. ') then replace (cl2.nome,'Sr. ','') 
  when startswith (cl2.nome, 'Sra. ') then replace (cl2.nome,'Sra. ','')
  when startswith (cl2.nome, 'Srta. ') then replace (cl2.nome,'Srta. ','')
  when startswith (cl2.nome, 'Dra. ') then replace (cl2.nome,'Dra. ', '')
  when startswith (cl2.nome, 'Dr. ') then replace (cl2.nome,'Dr. ','')
  else cl2.nome 
  end as nome_destinatario_normalizado,
  t.id as id_transacao,
  t.data,
  t.is_fraud,
  t.fraud_type
from transacoes_db.copper.clientes as cl
join transacoes_db.copper.contas as c
on cl.id = c.id_cliente
join transacoes_db.copper.transacoes as t
on c.id = t.id_conta_origem
join transacoes_db.copper.contas as c2
on t.id_conta_destino = c2.id
join transacoes_db.copper.clientes as cl2 -- join para chamar novamente a nome mas através do relacionamento cl2.id
on c2.id_cliente = cl2.id

In [0]:
%sql
select 
c.id_cliente,
c.id as id_conta,
c.agencia,
c.numero,
c.id_tipo_conta,
i.nome as Instituicao_Financeira,
c.estado_ibge,
c.municipio_ibge,
max(c.is_high_risk) as Maior_risco
from transacoes_db.copper.contas as c
left join transacoes_db.copper.instituicoes as i
on c.ispb_instituicao = i.ispb
group by All
order by 
maior_risco desc


Teste


In [0]:
%sql


In [0]:
print({len(df)})

In [0]:
%sql

SELECT
  -- Colunas da transação
  tx.valor AS valor_transacao,
  date(tx.data)  AS data_transacao,
  tx.id_conta_origem AS id_conta_pagador,
  tx.id_conta_destino AS id_conta_recebedor,
  
  tx.id_tipo_iniciacao_pix AS tipo_iniciacao_pix_id,
  tx.id_finalidade_pix AS finalidade_pix_id,
  tx.is_fraud AS transacao_fraudulenta,
  coalesce(tx.fraud_type,'Legitima') AS tipo_fraude,

  -- Pagador: Conta
  conta_orig.saldo AS pagador_saldo,
  conta_orig.aberta_em AS pagador_conta_aberta_em,
  conta_orig.id_tipo_conta AS pagador_tipo_conta_id,
  conta_orig.ispb_instituicao AS pagador_ispb_instituicao,
  
  conta_orig.estado_ibge AS pagador_estado_ibge, -- avalaiar se é um campo importante e se é possivel criar uma logica de fraude onde essas informações sejam relevantes
  conta_orig.municipio_ibge AS pagador_municipio_ibge, 

  -- Pagador: Cliente
  cliente_orig.id_natureza AS pagador_natureza_id,
  cliente_orig.nascido_em AS pagador_data_nascimento,
  cliente_orig.estado_ibge AS pagador_estado_ibge_cliente,
  cliente_orig.municipio_ibge AS pagador_municipio_ibge_cliente,

  -- Pagador: Instituição
  inst_orig.ispb AS pagador_instituicao_ispb,

  -- Pagador: Tipo de Conta
  tipo_conta_orig.id AS pagador_tipo_conta_id_ref,

  -- Pagador: Município
  mun_orig.codigo_ibge AS pagador_municipio_ibge_ref,

  -- Pagador: Natureza
  natureza_orig.id AS pagador_natureza_id_ref,

  -- Recebedor: Conta
  conta_dest.saldo AS recebedor_saldo,
  conta_dest.aberta_em AS recebedor_conta_aberta_em,
  conta_dest.id_tipo_conta AS recebedor_tipo_conta_id,
  conta_dest.estado_ibge AS recebedor_estado_ibge, -- avalaiar se é um campo importante e se é possivel criar uma logica de fraude onde essas informações sejam relevantes
  conta_dest.municipio_ibge AS recebedor_municipio_ibge,

  -- Recebedor: Cliente
  cliente_dest.id_natureza AS recebedor_natureza_id,
  cliente_dest.nascido_em AS recebedor_data_nascimento,

  cliente_dest.estado_ibge AS recebedor_estado_ibge_cliente, -- avalaiar se é um campo importante e se é possivel criar uma logica de fraude onde essas informações sejam relevantes
  cliente_dest.municipio_ibge AS recebedor_municipio_ibge_cliente,

  -- Recebedor: Tipo de Conta
  tipo_conta_dest.id AS recebedor_tipo_conta_id_ref,

  -- Recebedor: Município
  mun_dest.codigo_ibge AS recebedor_municipio_ibge_ref,

  -- Recebedor: Natureza
  natureza_dest.id AS recebedor_natureza_id_ref,


-- aumentar o numero de contas de triangulação 
  (
    ROW_NUMBER() OVER (
      PARTITION BY TX.id_conta_origem, tx.id_conta_destino
      ORDER BY   
        tx.data ASC 
    ) - 1

  ) as qtd_transacoes_pagador_recebedor,

    COUNT(tx.id_conta_origem) OVER (
      partition by tx.id_conta_origem, tx.data
    ) as qtd_transacoes_no_dia_pagador,
  
  COUNT(1) OVER (
      PARTITION BY tx.id_conta_destino, date(tx.data)
  ) AS recebedor_total_txs_no_dia
   
FROM
  transacoes_db.copper.transacoes AS tx

 left JOIN transacoes_db.copper.contas AS conta_orig
  ON tx.id_conta_origem = conta_orig.id
 left JOIN transacoes_db.copper.clientes AS cliente_orig
  ON conta_orig.id_cliente = cliente_orig.id
 left JOIN transacoes_db.copper.instituicoes AS inst_orig
  ON conta_orig.ispb_instituicao = inst_orig.ispb
 left JOIN transacoes_db.copper.tipos_conta AS tipo_conta_orig
  ON conta_orig.id_tipo_conta = tipo_conta_orig.id
 left JOIN transacoes_db.copper.municipios AS mun_orig
  ON cliente_orig.municipio_ibge = mun_orig.codigo_ibge
 left JOIN transacoes_db.copper.naturezas AS natureza_orig
  ON cliente_orig.id_natureza = natureza_orig.id

 left JOIN transacoes_db.copper.contas AS conta_dest
  ON tx.id_conta_destino = conta_dest.id
 left JOIN transacoes_db.copper.clientes AS cliente_dest
  ON conta_dest.id_cliente = cliente_dest.id
 left JOIN transacoes_db.copper.instituicoes AS inst_dest
  ON conta_dest.ispb_instituicao = inst_dest.ispb
 left JOIN transacoes_db.copper.tipos_conta AS tipo_conta_dest
  ON conta_dest.id_tipo_conta = tipo_conta_dest.id
 left JOIN transacoes_db.copper.municipios AS mun_dest
  ON cliente_dest.municipio_ibge = mun_dest.codigo_ibge
 left JOIN transacoes_db.copper.naturezas AS natureza_dest
  ON cliente_dest.id_natureza = natureza_dest.id

 left JOIN transacoes_db.copper.finalidade_pix AS finalidade_pix
  ON tx.id_finalidade_pix = finalidade_pix.id

where tx.is_fraud = 1
ORDER BY
  recebedor_total_txs_no_dia DESC

In [0]:
%sql
SELECT
  *
FROM
  system.access.table_lineage
WHERE
  target_table_full_name = 'transacoes_db.gold.transacoes_dataset'

In [0]:
%sql
-- =========================================================
-- 📘 DICIONÁRIO DE DADOS COM LINHAGEM AUTOMÁTICA (Unity Catalog)
-- Gera catálogo completo de colunas + origem (linhagem) + metadados
-- =========================================================

WITH colunas AS (
    SELECT
        c.TABLE_CATALOG,
        c.TABLE_SCHEMA,
        c.TABLE_NAME,
        c.COLUMN_NAME,
        c.DATA_TYPE,
        c.IS_NULLABLE,
        c.ORDINAL_POSITION
    FROM transacoes_db.information_schema.columns c
    UNION ALL
    SELECT
        c.TABLE_CATALOG,
        c.TABLE_SCHEMA,
        c.TABLE_NAME,
        c.COLUMN_NAME,
        c.DATA_TYPE,
        c.IS_NULLABLE,
        c.ORDINAL_POSITION
    FROM estatisticas_pix.information_schema.columns c
),

tabelas AS (
    SELECT
        t.TABLE_CATALOG,
        t.TABLE_SCHEMA,
        t.TABLE_NAME,
        t.TABLE_TYPE,
        t.TABLE_OWNER,
        t.COMMENT
    FROM transacoes_db.information_schema.tables t
    UNION ALL
    SELECT
        t.TABLE_CATALOG,
        t.TABLE_SCHEMA,
        t.TABLE_NAME,
        t.TABLE_TYPE,
        t.TABLE_OWNER,
        t.COMMENT
    FROM estatisticas_pix.information_schema.tables t
),

-- 🔹 Agrega a linhagem para cada tabela de destino
linhagem_agregada AS (
    SELECT
        target_table_full_name,
        collect_set(source_table_full_name) AS Tabelas_Origem,
        max(event_time) AS Linhagem_Ultima_Atualizacao_UTC
    FROM system.access.table_lineage
    WHERE target_table_catalog IN ('transacoes_db', 'estatisticas_pix')
    GROUP BY target_table_full_name
)

SELECT
    -- 1️⃣ Identificação da tabela
    T.TABLE_CATALOG AS Catalogo_Destino,
    T.TABLE_SCHEMA AS Esquema_Destino,
    T.TABLE_NAME AS Nome_Tabela_Destino,
    T.TABLE_TYPE AS Tipo_Objeto,

    -- 2️⃣ Detalhes da coluna
    C.COLUMN_NAME AS Nome_Coluna,
    C.DATA_TYPE AS Tipo_Dado,
    C.IS_NULLABLE AS Permite_Nulo,
    C.ORDINAL_POSITION AS Posicao,

    -- 3️⃣ Linhagem (origens agregadas)
    L.Tabelas_Origem,
    L.Linhagem_Ultima_Atualizacao_UTC,

    -- 4️⃣ Metadados adicionais
    T.TABLE_OWNER AS Proprietario_Tabela,
    T.COMMENT AS Comentario_Tabela

FROM colunas C
JOIN tabelas T
    ON C.TABLE_CATALOG = T.TABLE_CATALOG
    AND C.TABLE_SCHEMA = T.TABLE_SCHEMA
    AND C.TABLE_NAME = T.TABLE_NAME
LEFT JOIN linhagem_agregada L
    ON CONCAT(T.TABLE_CATALOG, '.', T.TABLE_SCHEMA, '.', T.TABLE_NAME) = L.target_table_full_name

WHERE
    T.TABLE_CATALOG IN ('transacoes_db', 'estatisticas_pix')
    AND T.TABLE_TYPE IN ('MANAGED', 'EXTERNAL', 'VIEW')  -- Inclui também as views (importante!)
    and t.TABLE_SCHEMA <> "information_schema"

ORDER BY
    Catalogo_Destino,
    Esquema_Destino,
    Nome_Tabela_Destino,
    Posicao;


In [0]:
%sql
SELECT
    -- Identificação da Tabela
    T.TABLE_CATALOG AS Catalogo,
    T.TABLE_SCHEMA AS Esquema,
    T.TABLE_NAME AS Nome_Tabela,
    T.TABLE_TYPE AS Tipo_Objeto, -- Ex: 'MANAGED', 'VIEW', 'EXTERNAL'
    
    -- Detalhes da Coluna
    C.COLUMN_NAME AS Nome_Coluna,
    C.DATA_TYPE AS Tipo_Dado,
    C.IS_NULLABLE AS Permite_Nulo,
    C.ORDINAL_POSITION AS Posicao,
    
    -- Metadados Adicionais
    T.TABLE_OWNER AS Proprietario_Tabela,
    T.COMMENT AS Comentario_Tabela
    
FROM
    -- Consulta a tabela de metadados do sistema (Unity Catalog)
    system.information_schema.tables T
JOIN
    system.information_schema.columns C
    ON T.TABLE_CATALOG = C.TABLE_CATALOG
    AND T.TABLE_SCHEMA = C.TABLE_SCHEMA
    AND T.TABLE_NAME = C.TABLE_NAME
WHERE
    T.TABLE_CATALOG = 'transacoes_db'
    AND T.TABLE_TYPE IN ('MANAGED', 'EXTERNAL') -- Filtra apenas tabelas de dados (exclui Views)
ORDER BY
    Esquema,
    Nome_Tabela,
    Posicao;

In [0]:
%sql

WITH tx_com_long AS (
  SELECT
    *,
    CAST(tx.data AS LONG) AS data_em_segundos
  FROM
    transacoes_db.gold.transacoes_balanced_model AS tx -- <<< 1. MUDANÇA AQUI (Usando a tabela V8)
),
/*
=============================================================================
 PASSO 2: CTE - Engenharia de Features de Tempo Real (Janelas Deslizantes)
 Aqui é onde a mágica acontece. Removemos as contagens de "dia inteiro"
 e as substituímos por contagens "na última X horas".
=============================================================================
*/
tx_features_realtime AS (
  SELECT
    tx.*,
    
    /* --- Features do Pagador (Janela de 24 horas) --- */
    
    -- Quantas TXs o Pagador fez nas últimas 24h?
    (COUNT(1) OVER (
      PARTITION BY id_conta_pagador
      ORDER BY data_em_segundos
      RANGE BETWEEN (24 * 3600) PRECEDING AND 1 PRECEDING -- (Olha 24h para trás, excluindo a si mesmo)
    )) AS pagador_txs_ultimas_24h,
    
    -- Quanto $ o Pagador movimentou nas últimas 24h?
    (COALESCE(SUM(valor_transacao) OVER (
      PARTITION BY id_conta_pagador
      ORDER BY data_em_segundos
      RANGE BETWEEN (24 * 3600) PRECEDING AND 1 PRECEDING
    ), 0)) AS pagador_valor_ultimas_24h,

    /* --- Features do Recebedor (Janela de 1 hora) --- */
    
    -- Quantas TXs o Recebedor recebeu na última 1h? (Sinal de "Sintoma")
    (COUNT(1) OVER (
      PARTITION BY id_conta_recebedor
      ORDER BY data_em_segundos
      RANGE BETWEEN 3600 PRECEDING AND 1 PRECEDING 
    )) AS recebedor_txs_ultima_1h,
    
    -- Quanto $ o Recebedor recebeu na última 1h?
    (COALESCE(SUM(valor_transacao) OVER (
      PARTITION BY id_conta_recebedor
      ORDER BY data_em_segundos
      RANGE BETWEEN 3600 PRECEDING AND 1 PRECEDING
    ), 0)) AS recebedor_valor_ultima_1h,

    /* --- Feature de Tempo (Lag) --- */
    
    -- Quanto tempo (em segundos) desde a última TX do pagador?
    (
      data_em_segundos - LAG(data_em_segundos, 1, 0) OVER (
        PARTITION BY id_conta_pagador
        ORDER BY data_em_segundos
      )
    ) AS pagador_segundos_desde_ultima_tx

  FROM
    tx_com_long tx
),
/*
=============================================================================
 PASSO 3: Junção Final (Seu SQL original, agora com features de tempo real)
 Juntamos as features de perfil (contas, clientes) com as
 novas features de tempo real que acabamos de criar.
=============================================================================
*/
SELECT
  -- Colunas da transação
  ft.valor_transacao,
  ft.data_transacao,
  ft.id_conta_pagador,
  ft.id_conta_recebedor,
  ft.tipo_iniciacao_pix_id,
  ft.finalidade_pix_id,
  ft.transacao_fraudulenta,
  ft.tipo_fraude,

  -- Pagador: Perfil
  conta_orig.saldo AS pagador_saldo,
  conta_orig.aberta_em AS pagador_conta_aberta_em,
  conta_orig.id_tipo_conta AS pagador_tipo_conta_id,
  cliente_orig.id_natureza AS pagador_natureza_id,
  cliente_orig.nascido_em AS pagador_data_nascimento,
  
  -- Recebedor: Perfil
  conta_dest.saldo AS recebedor_saldo,
  conta_dest.aberta_em AS recebedor_conta_aberta_em,
  conta_dest.id_tipo_conta AS recebedor_tipo_conta_id,
  cliente_dest.id_natureza AS recebedor_natureza_id,
  cliente_dest.nascido_em AS recebedor_data_nascimento,

  -- =======================================================
  -- <<< 2. NOVAS FEATURES DE TEMPO REAL >>>
  -- =======================================================
  ft.pagador_txs_ultimas_24h,
  ft.pagador_valor_ultimas_24h,
  ft.recebedor_txs_ultima_1h,
  ft.recebedor_valor_ultima_1h,
  ft.pagador_segundos_desde_ultima_tx

  /*
  REMOVIDAS (Eram features de Batch/Pós-Fato):
  - qtd_transacoes_pagador_recebedor
  - qtd_transacoes_no_dia_pagador
  - recebedor_total_txs_no_dia
  */

FROM
  tx_features_realtime AS ft
  
  -- Joins (exatamente como você tinha antes)
  LEFT JOIN transacoes_db.copper.contas AS conta_orig
    ON ft.id_conta_pagador = conta_orig.id
  LEFT JOIN transacoes_db.copper.clientes AS cliente_orig
    ON conta_orig.id_cliente = cliente_orig.id
  LEFT JOIN transacoes_db.copper.contas AS conta_dest
    ON ft.id_conta_recebedor = conta_dest.id
  LEFT JOIN transacoes_db.copper.clientes AS cliente_dest
    ON conta_dest.id_cliente = cliente_dest.id

-- <<< 3. MUDANÇA AQUI: REMOVA O 'WHERE' e 'ORDER BY' para o treino >>>
-- where tx.is_fraud = 1 (REMOVIDO)
-- ORDER BY recebedor_total_txs_no_dia DESC (REMOVIDO)